# Text Adventure Prompt Pipeline Explorer

This notebook recreates the 8-stage LLM prompt pipeline from the text adventure game, allowing you to iterate on each prompt in isolation.

## Pipeline Overview

**Per Turn Flow:**
1. **Director Intent Interpretation** - Convert user input to mutations
2. **Event Summarization** - Create canonical event lines from results
3. **Sensory Event Generation** - Generate environmental responses
4. **NPC Perception Filtering** - Decide what each NPC perceives
5. **NPC Situation Summarization** - Create present-tense context
6. **NPC Thought Generation** - Internal NPC reasoning
7. **NPC Action Generation** - Behavioral responses
8. **Narration** - Final storytelling layer

Each stage is isolated here so you can experiment with prompt modifications.

In [1]:
# Dependencies
import json
import openai
import os
from typing import Dict, List, Any
from pprint import pprint

# Configure OpenAI (adjust as needed)
openai.api_key = os.environ.get('OPENAI_API_KEY', 'your-key-here')

def call_llm(system_prompt: str, user_prompt: str, model: str = "gpt-4o-mini", max_tokens: int = 2000) -> str:
    """Helper function to call LLM with consistent parameters"""
    try:
        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=max_tokens,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def call_llm_json_schema(system_prompt: str, user_prompt: str, schema: Dict[str, Any], model: str = "gpt-4o-mini") -> str:
    """Helper function for JSON schema responses"""
    try:
        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"},
            max_tokens=2000,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f'{"error": "{str(e)}"}'

print("✅ Helper functions loaded")

✅ Helper functions loaded


## Sample World State Data

This recreates the game's world state structure for testing.

In [2]:
# Sample world state (mirrors the Go structs)
sample_world = {
    "location": "foyer",
    "inventory": [],
    "met_npcs": [],
    "locations": {
        "foyer": {
            "name": "foyer",
            "facts": ["A dusty entrance hall with checkered tiles", "Dim light filters through grimy windows"],
            "exits": {"north": "study", "east": "library", "west": "kitchen"}
        },
        "library": {
            "name": "library", 
            "facts": ["Tall bookshelves line the walls", "A reading table sits in the center"],
            "exits": {"west": "foyer"}
        },
        "study": {
            "name": "study",
            "facts": ["A wooden desk sits against the wall", "Papers are scattered about"],
            "exits": {"south": "foyer"}
        },
        "kitchen": {
            "name": "kitchen",
            "facts": ["Old cabinets and a rusty sink"],
            "exits": {"east": "foyer"}
        }
    },
    "npcs": {
        "elena": {
            "location": "library",
            "personality": "cautious, observant, struggling with disorientation",
            "backstory": "recently awakened in this strange place with no memory of how she got here or who she was before",
            "memories": [
                "woke up somewhere unfamiliar",
                "has no memory of her past", 
                "feeling disoriented and cautious"
            ],
            "facts": [],
            "recent_thoughts": [],
            "recent_actions": [],
            "inventory": [],
            "description": "someone"
        }
    }
}

# Sample conversation history
sample_history = [
    "Player: You find yourself in a dusty foyer.",
    "Player: look around",
    "System: You see exits to the north, east, and west."
]

def build_world_context(world: Dict[str, Any], history: List[str], npc_id: str = "") -> str:
    """Build world context string like the Go version"""
    context = f"CURRENT LOCATION: {world['location']}\n"
    context += f"PLAYER INVENTORY: {', '.join(world['inventory']) if world['inventory'] else 'empty'}\n\n"
    
    # Location details
    current_loc = world['locations'][world['location']]
    context += f"LOCATION DETAILS ({world['location']})::\n"
    for fact in current_loc['facts']:
        context += f"- {fact}\n"
    
    context += f"\nEXITS: {', '.join(f'{direction} to {dest}' for direction, dest in current_loc['exits'].items())}\n\n"
    
    # NPCs
    current_npcs = [npc for npc_name, npc in world['npcs'].items() if npc['location'] == world['location']]
    if current_npcs:
        context += "NPCs HERE:\n"
        for npc in current_npcs:
            context += f"- {npc['description']}\n"
    
    # Add history if provided
    if history:
        context += f"\nRECENT CONVERSATION:\n"
        for line in history[-5:]:  # Last 5 lines
            context += f"- {line}\n"
    
    return context

print("✅ Sample world data loaded")
print("Sample world context:")
print(build_world_context(sample_world, sample_history))

✅ Sample world data loaded
Sample world context:
CURRENT LOCATION: foyer
PLAYER INVENTORY: empty

LOCATION DETAILS (foyer)::
- A dusty entrance hall with checkered tiles
- Dim light filters through grimy windows

EXITS: north to study, east to library, west to kitchen


RECENT CONVERSATION:
- Player: You find yourself in a dusty foyer.
- Player: look around
- System: You see exits to the north, east, and west.



## Stage 1: Director Intent Interpretation

The Director LLM converts natural language user input into structured mutations.

In [3]:
def build_director_prompt(world: Dict[str, Any], history: List[str], acting_npc_id: str = "") -> str:
    """Build the director system prompt"""
    
    tool_descriptions = """move_player(location: string) - Move the player to a specific location
move_npc(npc_id: string, location: string) - Move an NPC to a specific location
transfer_item(item: string, from_location: string, to_location: string) - Move an item between locations or entities
add_to_inventory(item: string) - Add an item from current location to player's inventory
remove_from_inventory(item: string) - Remove an item from player's inventory to current location
mark_npc_as_met(npc_id: string) - Mark that the player has met and learned an NPC's name"""
    
    action_label = "Player action" if not acting_npc_id else f"NPC {acting_npc_id.upper()} ACTION"
    
    if acting_npc_id:
        movement_guideline = f"- Movement: use move_npc with npc_id=\"{acting_npc_id}\"."
        pickup_guidelines = f"- Pick up item: use transfer_item from location → {acting_npc_id}.\n- If NPC introduces themselves: use mark_npc_as_met with npc_id=\"{acting_npc_id}\"."
        example_destination = acting_npc_id
    else:
        movement_guideline = "- Movement: use move_player."
        pickup_guidelines = "- Pick up item: use transfer_item from location → player, then add_to_inventory.\n- If meeting someone who gives their name: use mark_npc_as_met with their npc_id."
        example_destination = "player"
    
    world_context = build_world_context(world, history, acting_npc_id)
    
    return f"""You are the Director of a text adventure game. Generate only the world mutations required to fulfill the user's intent.

<available_tools>
{tool_descriptions}
</available_tools>

<context>
{world_context}
</context>

<guidelines>
- Interpret the {action_label} and produce only necessary mutations using the available tools.
- Output strictly as a JSON object: {{"mutations": [ ... ]}} — no extra text.
- Be conservative; avoid speculative or unrelated changes.
{movement_guideline}
{pickup_guidelines}
- Drop item: remove_from_inventory, then transfer_item to current location.
- Examine/look at environment: usually no mutations needed.
- Examine/look at NPCs or specific items: may need mutations to trigger detailed descriptions or NPC reactions.
- NPCs may only affect items at their location or move themselves.
</guidelines>

<example_output>
{{"mutations": [
  {{"tool": "move_player", "args": {{"location": "kitchen"}}}},
  {{"tool": "transfer_item", "args": {{"item": "key", "from_location": "foyer", "to_location": "{example_destination}"}}}}
]}}
</example_output>"""

# Test Stage 1: Director Intent Interpretation
def test_director_stage(user_input: str, world: Dict[str, Any], history: List[str]):
    print(f"🎯 STAGE 1: Director Intent Interpretation")
    print(f"User Input: '{user_input}'")
    print("\n" + "="*60)
    
    system_prompt = build_director_prompt(world, history)
    user_prompt = f"Player action: {user_input}"
    
    print("\n📋 SYSTEM PROMPT:")
    print(system_prompt)
    print("\n📝 USER PROMPT:")
    print(user_prompt)
    
    response = call_llm_json_schema(system_prompt, user_prompt, {})
    
    print("\n🤖 LLM RESPONSE:")
    try:
        parsed = json.loads(response)
        print(json.dumps(parsed, indent=2))
        return parsed
    except json.JSONDecodeError:
        print(f"Raw response: {response}")
        return {"mutations": []}

# Example test
mutations_result = test_director_stage("go to the library", sample_world, sample_history)

🎯 STAGE 1: Director Intent Interpretation
User Input: 'go to the library'


📋 SYSTEM PROMPT:
You are the Director of a text adventure game. Generate only the world mutations required to fulfill the user's intent.

<available_tools>
move_player(location: string) - Move the player to a specific location
move_npc(npc_id: string, location: string) - Move an NPC to a specific location
transfer_item(item: string, from_location: string, to_location: string) - Move an item between locations or entities
add_to_inventory(item: string) - Add an item from current location to player's inventory
remove_from_inventory(item: string) - Remove an item from player's inventory to current location
mark_npc_as_met(npc_id: string) - Mark that the player has met and learned an NPC's name
</available_tools>

<context>
CURRENT LOCATION: foyer
PLAYER INVENTORY: empty

LOCATION DETAILS (foyer)::
- A dusty entrance hall with checkered tiles
- Dim light filters through grimy windows

EXITS: north to study, east to 

## Stage 2: Event Summarization

Creates human-readable event lines from the mutations and their results.

In [4]:
def test_event_summarization_stage(user_input: str, npc_id: str, old_world: Dict[str, Any], new_world: Dict[str, Any], successes: List[str], failures: List[str]):
    print(f"📝 STAGE 2: Event Summarization")
    print("\n" + "="*60)
    
    actor = "PLAYER" if not npc_id else npc_id.upper()
    
    world_delta_hint = ""
    if old_world['location'] != new_world['location']:
        world_delta_hint = f"Location changed: {old_world['location']} -> {new_world['location']}"
    
    user_prompt_parts = [
        f"ACTOR: {actor}",
        f"INPUT: {user_input}"
    ]
    
    if successes:
        user_prompt_parts.append(f"SUCCESSES:\n{chr(10).join(successes)}")
    
    if failures:
        user_prompt_parts.append(f"FAILURES:\n{chr(10).join(failures)}")
    
    if world_delta_hint:
        user_prompt_parts.append(f"WORLD HINT: {world_delta_hint}")
    
    user_prompt = "\n".join(user_prompt_parts)
    
    schema = {
        "type": "object",
        "properties": {
            "events": {
                "type": "array",
                "items": {"type": "string"},
                "description": "Array of short, human-readable lines describing what actually happened this turn"
            }
        },
        "required": ["events"],
        "additionalProperties": False
    }
    
    system_prompt = """You summarize the outcome of a single game turn.
Output the events as an array of short, human-readable lines describing what actually happened this turn.
Use present tense. Do not invent events. It's OK if some lines describe attempts that didn't change state (like examining)."""
    
    print("\n📋 SYSTEM PROMPT:")
    print(system_prompt)
    print("\n📝 USER PROMPT:")
    print(user_prompt)
    
    response = call_llm_json_schema(system_prompt, user_prompt, schema)
    
    print("\n🤖 LLM RESPONSE:")
    try:
        parsed = json.loads(response)
        print(json.dumps(parsed, indent=2))
        return parsed.get('events', [])
    except json.JSONDecodeError:
        print(f"Raw response: {response}")
        return []

# Example test - simulate successful movement
new_world = sample_world.copy()
new_world['location'] = 'library'

event_lines = test_event_summarization_stage(
    "go to the library",
    "",
    sample_world,
    new_world, 
    ["Player moved from foyer to library"],
    []
)

📝 STAGE 2: Event Summarization


📋 SYSTEM PROMPT:
You summarize the outcome of a single game turn.
Output the events as an array of short, human-readable lines describing what actually happened this turn.
Use present tense. Do not invent events. It's OK if some lines describe attempts that didn't change state (like examining).

📝 USER PROMPT:
ACTOR: PLAYER
INPUT: go to the library
SUCCESSES:
Player moved from foyer to library
WORLD HINT: Location changed: foyer -> library


ValueError: Invalid format specifier

## Stage 3: Sensory Event Generation

Generates environmental audio events that other NPCs might perceive.

In [ ]:
def test_sensory_event_stage(user_input: str, world: Dict[str, Any], mutation_results: List[str]):
    print(f"👂 STAGE 3: Sensory Event Generation")
    print("\n" + "="*60)
    
    system_prompt = """You are a sensory event generator for a text adventure game. Generate descriptive auditory events for player actions.

Rules:
- Generate only ONE self-contained event per action
- Events represent what happened in THIS turn only - not ongoing states
- ONLY describe what can actually be HEARD - no visual details or object identification
- Use complete descriptions: "someone walked from foyer to library" not just "footsteps"
- Use objective third-person descriptions: "someone shouted", "door creaking", "rustling sounds"
- Sounds cannot identify specific objects - describe the sound, not what caused it
- Capture actual content when relevant: include spoken words, but not visual details
- Volume levels: "quiet", "moderate", "loud"
- Quiet actions like "look around" = no events

Return JSON only:
{
  "auditory_events": [
    {
      "type": "auditory", 
      "description": "someone shouted 'Elena, I'm here!'",
      "location": "foyer",
      "volume": "loud"
    }
  ]
}

If no sound, return empty auditory_events array."""
    
    context_parts = [
        f"USER ACTION: {user_input}",
        f"CURRENT LOCATION: {world['location']}"
    ]
    
    if mutation_results:
        context_parts.append(f"MUTATION RESULTS: {', '.join(mutation_results)}")
    
    user_prompt = "\n".join(context_parts)
    
    print("\n📋 SYSTEM PROMPT:")
    print(system_prompt)
    print("\n📝 USER PROMPT:")
    print(user_prompt)
    
    response = call_llm_json_schema(system_prompt, user_prompt, {})
    
    print("\n🤖 LLM RESPONSE:")
    try:
        parsed = json.loads(response)
        print(json.dumps(parsed, indent=2))
        return parsed.get('auditory_events', [])
    except json.JSONDecodeError:
        print(f"Raw response: {response}")
        return []

# Example test
sensory_events = test_sensory_event_stage(
    "shout 'Hello! Is anyone there?'",
    sample_world,
    ["Player shouted in foyer"]
)

## Stage 4: NPC Perception Filtering

Each NPC's LLM decides what they can perceive from the world events.

In [ ]:
def test_npc_perception_stage(npc_id: str, world: Dict[str, Any], world_event_lines: List[str]):
    print(f"👁️ STAGE 4: NPC Perception Filtering ({npc_id})")
    print("\n" + "="*60)
    
    if not world_event_lines:
        print("No world events to process")
        return []
    
    world_context = build_world_context(world, [], npc_id)
    
    user_prompt_parts = [
        f"NPC: {npc_id}",
        "",
        f"WORLD SNAPSHOT (for reasoning):\n{world_context}",
        "",
        f"EVENT LINES:\n{chr(10).join(world_event_lines)}"
    ]
    
    user_prompt = "\n".join(user_prompt_parts)
    
    schema = {
        "type": "object",
        "properties": {
            "events": {
                "type": "array",
                "items": {"type": "string"},
                "description": "Array of perceived event strings"
            }
        },
        "required": ["events"],
        "additionalProperties": False
    }
    
    system_prompt = """You decide what an NPC perceives in a text adventure.
Given a world snapshot and a list of canonical event lines from this turn, select only the lines the NPC could plausibly perceive.
Rules:
- Return a JSON object with an "events" array containing strings strictly chosen from the provided event lines.
- Do not invent or paraphrase; copy the exact lines that would be perceived.
- Event lines may include tags of the form "Actor@location: ...". Prefer selecting lines where the location matches the NPC's current room.
- Consider location, proximity, and what could be seen or heard (e.g., speech may carry to nearby rooms; be conservative).
- If nothing is perceived, return {"events": []}"""
    
    print("\n📋 SYSTEM PROMPT:")
    print(system_prompt)
    print("\n📝 USER PROMPT:")
    print(user_prompt)
    
    response = call_llm_json_schema(system_prompt, user_prompt, schema)
    
    print("\n🤖 LLM RESPONSE:")
    try:
        parsed = json.loads(response)
        print(json.dumps(parsed, indent=2))
        return parsed.get('events', [])
    except json.JSONDecodeError:
        print(f"Raw response: {response}")
        return []

# Example test - Elena in library perceiving events from foyer
sample_events = [
    "Player@foyer: go to the library",
    "Player moved from foyer to library",
    "Footsteps approach from the foyer"
]

perceived_events = test_npc_perception_stage("elena", sample_world, sample_events)

## Stage 5: NPC Situation Summarization

Lightweight present-tense context bridging "what just happened" to "what's happening now".

In [ ]:
def test_npc_situation_stage(npc_id: str, world: Dict[str, Any], perceived_lines: List[str]):
    print(f"🌍 STAGE 5: NPC Situation Summarization ({npc_id})")
    print("\n" + "="*60)
    
    world_context = build_world_context(world, [], npc_id)
    
    user_prompt_parts = [
        f"<world_context>\n{world_context.strip()}\n</world_context>",
        "",
        "<perceived_events>"
    ]
    
    for event in perceived_lines:
        user_prompt_parts.append(f"- {event.strip()}")
    
    user_prompt_parts.append("</perceived_events>")
    
    user_prompt = "\n".join(user_prompt_parts)
    
    system_prompt = """Summarize the immediate situation in 1-2 short sentences in present tense.
Use only the provided world_context and perceived_events.
Be concrete and neutral. No invention beyond those details."""
    
    print("\n📋 SYSTEM PROMPT:")
    print(system_prompt)
    print("\n📝 USER PROMPT:")
    print(user_prompt)
    
    response = call_llm(system_prompt, user_prompt, max_tokens=1000)
    
    print("\n🤖 LLM RESPONSE:")
    print(response)
    
    return response.strip()

# Example test
situation = test_npc_situation_stage("elena", sample_world, perceived_events)

## Stage 6: NPC Thought Generation

Internal NPC reasoning based on their character and what they've perceived.

In [ ]:
def build_npc_thoughts_prompt(npc_id: str, npc_data: Dict[str, Any]) -> str:
    """Build the NPC thoughts system prompt with XML structure"""
    
    prompt_parts = [f"You are {npc_id}. Generate a single internal thought based on your current situation."]
    prompt_parts.append("\n<character>")
    prompt_parts.append(f"- name: {npc_id}")
    
    if npc_data.get('personality', '').strip():
        prompt_parts.append(f"- personality: {npc_data['personality']}")
    
    if npc_data.get('backstory', '').strip():
        prompt_parts.append(f"- backstory: {npc_data['backstory']}")
    
    if npc_data.get('memories', []):
        prompt_parts.append("- core_memories:")
        for memory in npc_data['memories']:
            prompt_parts.append(f"  - {memory}")
    
    prompt_parts.append("</character>\n")
    
    prompt_parts.append("<recent_memory>")
    
    if npc_data.get('recent_thoughts', []):
        prompt_parts.append("- thoughts:")
        for thought in npc_data['recent_thoughts']:
            prompt_parts.append(f"  - {thought}")
    
    if npc_data.get('recent_actions', []):
        prompt_parts.append("- actions:")
        for action in npc_data['recent_actions']:
            prompt_parts.append(f"  - {action}")
    
    prompt_parts.append("</recent_memory>\n")
    
    prompt_parts.append("""<style>
- one line only
- present tense; natural and practical
- base only on world_context and perceived_events
- no quotes; no role labels; no narration
- avoid repeating identical prior thoughts; build on change
- it's fine to be uncertain or to simply observe; don't force a plan
</style>""")
    
    return "\n".join(prompt_parts)

def build_npc_thoughts_user_prompt(world_context: str, perceived_lines: List[str], situation: str) -> str:
    """Build the user prompt for NPC thoughts"""
    
    parts = [f"<world_context>\n{world_context.strip()}\n</world_context>\n"]
    
    if situation.strip():
        parts.append(f"<situation>\n{situation.strip()}\n</situation>\n")
    
    parts.append("<perceived_events>")
    for event in perceived_lines:
        parts.append(f"- {event.strip()}")
    parts.append("</perceived_events>")
    
    return "\n".join(parts)

def test_npc_thoughts_stage(npc_id: str, world: Dict[str, Any], perceived_lines: List[str], situation: str):
    print(f"🧠 STAGE 6: NPC Thought Generation ({npc_id})")
    print("\n" + "="*60)
    
    npc_data = world['npcs'][npc_id]
    world_context = build_world_context(world, [], npc_id)
    
    system_prompt = build_npc_thoughts_prompt(npc_id, npc_data)
    user_prompt = build_npc_thoughts_user_prompt(world_context, perceived_lines, situation)
    
    print("\n📋 SYSTEM PROMPT:")
    print(system_prompt)
    print("\n📝 USER PROMPT:")
    print(user_prompt)
    
    response = call_llm(system_prompt, user_prompt, max_tokens=2000)
    
    print("\n🤖 LLM RESPONSE:")
    print(response)
    
    return response.strip()

# Example test
npc_thoughts = test_npc_thoughts_stage("elena", sample_world, perceived_events, situation)

## Stage 7: NPC Action Generation

Generate behavioral responses based on NPC thoughts and world state.

In [ ]:
def build_npc_action_prompt(npc_id: str, npc_thoughts: str, npc_data: Dict[str, Any]) -> str:
    """Build the NPC action system prompt"""
    
    memory_context = ""
    if npc_data.get('recent_actions', []):
        actions_list = npc_data['recent_actions']
        memory_context = f"\n\nYour recent actions: {actions_list}\nDon't repeat the same action unless something has changed."
    
    personality_context = ""
    if npc_data.get('personality', '').strip():
        personality_context = f"- Personality: {npc_data['personality']}\n"
    
    backstory_context = ""
    if npc_data.get('backstory', '').strip():
        backstory_context = f"- Background: {npc_data['backstory']}\n"
    
    return f"""You are {npc_id}. React realistically to your current situation — you don't have to "pick an action" every turn.

Your character:
- Name: {npc_id}
{personality_context}{backstory_context}- You act naturally based on what you've noticed and what you're thinking
- You can move between rooms, talk to people, interact with objects, or simply pause to observe or think
- Only act if it makes sense right now; it's valid to call out, look around, or do nothing

Your current thoughts: "{npc_thoughts}"{memory_context}

Based on your thoughts and the world state, what do you want to do? You can:
- Move to a different room (e.g., "go to kitchen") 
- Say something (e.g., "say Hello there!")
- Pick up an item (e.g., "take key")
- Look around or examine something (e.g., "look around", "examine desk")
- Call out (e.g., "say Is someone there?")
- Do nothing (return empty string)

Return only a brief action statement, or an empty string if you don't want to act."""

def build_npc_world_context_with_perceptions(npc_id: str, world: Dict[str, Any], perceived_lines: List[str]) -> str:
    """Build world context with perceived events"""
    
    base_context = build_world_context(world, [], npc_id)
    
    if not perceived_lines:
        return base_context
    
    perceived_section = "PERCEIVED EVENTS:\n"
    for line in perceived_lines:
        perceived_section += f"- {line.strip()}\n"
    perceived_section += "\n"
    
    # Insert perceived events before RECENT CONVERSATION if it exists
    if "RECENT CONVERSATION:" in base_context:
        return base_context.replace("RECENT CONVERSATION:", perceived_section + "RECENT CONVERSATION:")
    
    return base_context + perceived_section

def test_npc_action_stage(npc_id: str, npc_thoughts: str, world: Dict[str, Any], perceived_lines: List[str]):
    print(f"🎭 STAGE 7: NPC Action Generation ({npc_id})")
    print("\n" + "="*60)
    
    if not npc_thoughts.strip():
        print("No thoughts provided - skipping action generation")
        return ""
    
    npc_data = world['npcs'][npc_id]
    world_context = build_npc_world_context_with_perceptions(npc_id, world, perceived_lines)
    
    system_prompt = build_npc_action_prompt(npc_id, npc_thoughts, npc_data)
    user_prompt = world_context
    
    print("\n📋 SYSTEM PROMPT:")
    print(system_prompt)
    print("\n📝 USER PROMPT:")
    print(user_prompt)
    
    response = call_llm(system_prompt, user_prompt, max_tokens=2000)
    
    print("\n🤖 LLM RESPONSE:")
    print(response)
    
    return response.strip()

# Example test
npc_action = test_npc_action_stage("elena", npc_thoughts, sample_world, perceived_events)

## Stage 8: Narration

Final storytelling layer that presents the results to the player.

In [ ]:
def build_narration_prompt(action_context: str, mutation_results: List[str], world_event_lines: List[str]) -> str:
    """Build the narration system prompt"""
    
    action_and_mutation_context = ""
    if action_context:
        action_and_mutation_context = f"\n\nACTION THAT JUST OCCURRED:\n{action_context}"
        
        if mutation_results:
            action_and_mutation_context += "\n\nWORLD CHANGES:\n" + "\n".join(mutation_results)
        
        action_and_mutation_context += "\n\nNarrate the consequences and results of this action."
    
    events_context = ""
    if world_event_lines:
        events_context = "\n\nWORLD EVENTS FOR THIS TURN:\n"
        for line in world_event_lines:
            events_context += f"- {line.strip()}\n"
    
    return f"""You are the narrator for an LLM-powered narrative text game. This is collaborative story-building - your role is to create an engaging story for the player to enjoy.

IMPORTANT: You narrate strictly from the player's perspective. You only know what the player can directly observe, experience, or interact with. You have no omniscient knowledge about hidden details, background information, or things the player hasn't encountered.

You see "Established Facts" for locations, items, and characters. These are canonical details that the player has already observed through previous narrations. Build naturally from these without contradicting them.

If the existing facts provide enough context for the current moment, work with what's established. You may add new details when the story naturally calls for them, but only describe what the player would actually notice or experience in this moment.

Your descriptions become part of the permanent world canon - anything you narrate becomes an established fact that the player has observed.

Rules:
- Base narration on the provided world events and world changes below. Focus on what happened as a result of the player's action.
- Use present tense. Write 2-4 sentences that create a good story experience.
- Only describe what the player can directly perceive through their senses or actions.
- If an event contains speech, render the words as quoted dialogue.
- If an action failed (as indicated by events/changes), briefly note why without giving advice.
- If there are no events or changes, write a single short beat that reflects the quiet or lack of change.

Only use information from the inputs below:{action_and_mutation_context}{events_context}"""

def test_narration_stage(action_context: str, mutation_results: List[str], world_event_lines: List[str]):
    print(f"📖 STAGE 8: Narration")
    print("\n" + "="*60)
    
    system_prompt = build_narration_prompt(action_context, mutation_results, world_event_lines)
    user_prompt = "Generate narration for the events described above."
    
    print("\n📋 SYSTEM PROMPT:")
    print(system_prompt)
    print("\n📝 USER PROMPT:")
    print(user_prompt)
    
    response = call_llm(system_prompt, user_prompt, max_tokens=2000)
    
    print("\n🤖 LLM RESPONSE:")
    print(response)
    
    return response.strip()

# Example test
final_narration = test_narration_stage(
    "PLAYER: go to the library",
    ["Player moved from foyer to library"],
    [
        "Player@foyer: go to the library",
        "Player moved from foyer to library",
        "Footsteps echo from foyer to library"
    ]
)

## Complete Pipeline Test

Run the entire 8-stage pipeline with a single user input to see the full flow.

In [ ]:
def run_complete_pipeline(user_input: str, world: Dict[str, Any], history: List[str]):
    print("🚀 COMPLETE PIPELINE TEST")
    print(f"User Input: '{user_input}'")
    print("\n" + "="*80)
    
    # Stage 1: Director Intent Interpretation
    mutations_result = test_director_stage(user_input, world, history)
    print("\n\n")
    
    # Simulate mutation execution results
    successes = ["Player moved from foyer to library"]
    failures = []
    new_world = world.copy()
    new_world['location'] = 'library'
    
    # Stage 2: Event Summarization
    event_lines = test_event_summarization_stage(user_input, "", world, new_world, successes, failures)
    print("\n\n")
    
    # Stage 3: Sensory Event Generation
    sensory_events = test_sensory_event_stage(user_input, world, successes)
    print("\n\n")
    
    # Stage 4: NPC Perception Filtering (for Elena)
    perceived_events = test_npc_perception_stage("elena", new_world, event_lines)
    print("\n\n")
    
    # Stage 5: NPC Situation Summarization 
    situation = test_npc_situation_stage("elena", new_world, perceived_events)
    print("\n\n")
    
    # Stage 6: NPC Thought Generation
    npc_thoughts = test_npc_thoughts_stage("elena", new_world, perceived_events, situation)
    print("\n\n")
    
    # Stage 7: NPC Action Generation
    npc_action = test_npc_action_stage("elena", npc_thoughts, new_world, perceived_events)
    print("\n\n")
    
    # Stage 8: Narration
    final_narration = test_narration_stage(
        f"PLAYER: {user_input}",
        successes,
        event_lines
    )
    
    print("\n\n" + "="*80)
    print("🏁 PIPELINE COMPLETE")
    print("\n📋 SUMMARY:")
    print(f"- Mutations generated: {len(mutations_result.get('mutations', []))}")
    print(f"- Event lines: {len(event_lines)}")
    print(f"- Elena perceived: {len(perceived_events)} events")
    print(f"- Elena's thought: '{npc_thoughts}'")
    print(f"- Elena's action: '{npc_action}'")
    print(f"- Final narration: '{final_narration[:100]}...'")

# Run complete test
run_complete_pipeline("go to the library", sample_world, sample_history)

## Experimentation Section

Use the cells below to experiment with individual stages or test different scenarios.

In [ ]:
# Experiment with different user inputs
experimental_inputs = [
    "look around",
    "shout 'Elena! Where are you?'",
    "examine the dusty tiles",
    "go north to the study",
    "say hello to anyone nearby"
]

for test_input in experimental_inputs:
    print(f"\n🧪 TESTING: '{test_input}'")
    print("-" * 50)
    mutations = test_director_stage(test_input, sample_world, sample_history)
    print(f"Generated {len(mutations.get('mutations', []))} mutations")
    print()

In [ ]:
# Experiment with NPC behavior in different scenarios
test_scenarios = [
    {
        "perceived_events": ["Player@library: Hello Elena!"],
        "description": "Player greets Elena directly"
    },
    {
        "perceived_events": ["Loud crash from foyer", "Glass shattering sounds"],
        "description": "Alarming sounds from adjacent room"
    },
    {
        "perceived_events": [],
        "description": "Complete silence - no events"
    }
]

for scenario in test_scenarios:
    print(f"\n🎭 SCENARIO: {scenario['description']}")
    print("-" * 50)
    
    situation = test_npc_situation_stage("elena", sample_world, scenario['perceived_events'])
    thoughts = test_npc_thoughts_stage("elena", sample_world, scenario['perceived_events'], situation)
    action = test_npc_action_stage("elena", thoughts, sample_world, scenario['perceived_events'])
    
    print(f"\n💭 Result - Thought: '{thoughts}'")
    print(f"🎯 Result - Action: '{action}'")
    print()